与えられた文の係り受け木を有向グラフとして可視化せよ．可視化には，係り受け木をDOT言語に変換し，Graphvizを用いるとよい．また，Pythonから有向グラフを直接的に可視化するには，pydotを使うとよい．

In [3]:
from pprint import pprint
import re
from subprocess import run, PIPE

import pydot

In [4]:
# 区切り文字
separator = re.compile('\t|,')

# 係り受け
dependancy = re.compile(r'''(?:\*\s\d+\s) # キャプチャ対象外
                            (-?\d+)       # 数字(係り先)
                          ''', re.VERBOSE)

In [6]:
text = input('テキスト入力してください')

# 初期値
if len(text) == 0:
    text = '言ってあったか言ってなかったかどっちだったかちゃんと覚えていないけど、確かこの間手巻きパーティをやった時にちょこっと言った気がしなくもなきにしもあらずで多分言ったんじゃないかな、とココまで考えてみたけど、まあ言ってようが言っていまいがそこまで問題ないよね、と思うに至った次第です。'

cmd = 'echo {} | cabocha -f1'.format(text)
proc = run(cmd, shell=True, stdout=PIPE, stderr=PIPE)
print(proc.stdout.decode('UTF-8'))

テキスト入力してください 


* 0 1D 0/4 0.285960
言っ	動詞,自立,*,*,五段・ワ行促音便,連用タ接続,言う,イッ,イッ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
あっ	動詞,非自立,*,*,五段・ラ行,連用タ接続,ある,アッ,アッ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
か	助詞,副助詞／並立助詞／終助詞,*,*,*,*,か,カ,カ
* 1 4D 0/4 2.230543
言っ	動詞,自立,*,*,五段・ワ行促音便,連用タ接続,言う,イッ,イッ
て	動詞,非自立,*,*,一段,未然形,てる,テ,テ
なかっ	助動詞,*,*,*,特殊・ナイ,連用タ接続,ない,ナカッ,ナカッ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
か	助詞,副助詞／並立助詞／終助詞,*,*,*,*,か,カ,カ
* 2 4D 0/3 2.418727
どっち	名詞,代名詞,一般,*,*,*,どっち,ドッチ,ドッチ
だっ	助動詞,*,*,*,特殊・ダ,連用タ接続,だ,ダッ,ダッ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
か	助詞,副助詞／並立助詞／終助詞,*,*,*,*,か,カ,カ
* 3 4D 0/0 2.560443
ちゃんと	副詞,一般,*,*,*,*,ちゃんと,チャント,チャント
* 4 19D 0/4 0.386971
覚え	動詞,自立,*,*,一段,連用形,覚える,オボエ,オボエ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
い	動詞,非自立,*,*,一段,未然形,いる,イ,イ
ない	助動詞,*,*,*,特殊・ナイ,基本形,ない,ナイ,ナイ
けど	助詞,接続助詞,*,*,*,*,けど,ケド,ケド
、	記号,読点,*,*,*,*,、,、,、
* 5 7D 0/0 0.261783
確か	副詞,一般,*,*,*,*,確か,タシカ,タシカ
* 6 7D 3/4 1.660965
この間	名詞,副詞可能,*,*,*,*,この間,コノカン,コノカン
手	名詞,一般,*,*,*,*,手,テ,テ
巻き	名詞,接尾,一般,*,*,*,巻き,マキ,マキ
パーティ	名詞,サ変接続,*,*,*,*,パーティ,パーティ,パーティ
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
* 7 8D 0/1 2.453394
やっ	動詞,自立,*,*,五段・ラ行,連用タ接続,やる,ヤッ,

In [7]:
class Chunk:
    def __init__(self, phrase, dst):
        self.phrase = phrase
        self.dst = dst  # 係り先文節インデックス番号

In [8]:
phrase = ''
chunks = []
for line in proc.stdout.decode('UTF-8').splitlines():
    dependancies = dependancy.match(line)
    
    # EOSまたは係り受け解析結果でない場合(EOSに改行はつかない点に注意)
    if not (line == 'EOS' or dependancies):
        #タブとカンマで分割
        cols = separator.split(line)
        phrase += cols[0] # 表層形(surface)

    # EOSまたは係り受け解析結果で、形態素解析結果がある場合
    elif phrase != '':
        chunks.append(Chunk(phrase, dst))
        phrase = ''

    # 係り受け結果の場合
    if dependancies:
        dst = int(dependancies.group(1))

In [9]:
#for chunk in chunks:
#    print(chunk.__dict__)
    
# 係り先があるものをpydotに渡す形式に変更
edges = []
for i, chunk in enumerate(chunks):
    if chunk.dst != -1 and \
       chunk.phrase != '' and \
       chunks[chunk.dst].phrase != '':
        edges.append(((i, chunk.phrase), (chunk.dst, chunks[chunk.dst].phrase)))    

{'phrase': '言ってあったか', 'dst': 1}
{'phrase': '言ってなかったか', 'dst': 4}
{'phrase': 'どっちだったか', 'dst': 4}
{'phrase': 'ちゃんと', 'dst': 4}
{'phrase': '覚えていないけど、', 'dst': 19}
{'phrase': '確か', 'dst': 7}
{'phrase': 'この間手巻きパーティを', 'dst': 7}
{'phrase': 'やった', 'dst': 8}
{'phrase': '時に', 'dst': 10}
{'phrase': 'ちょこっと', 'dst': 10}
{'phrase': '言った', 'dst': 11}
{'phrase': '気が', 'dst': 12}
{'phrase': 'しなくも', 'dst': 14}
{'phrase': 'なき', 'dst': 14}
{'phrase': 'にしも', 'dst': 15}
{'phrase': 'あらずで', 'dst': 17}
{'phrase': '多分', 'dst': 17}
{'phrase': '言ったんじゃないかな、と', 'dst': 19}
{'phrase': 'ココまで', 'dst': 19}
{'phrase': '考えてみたけど、', 'dst': 28}
{'phrase': 'まあ', 'dst': 21}
{'phrase': '言ってようが', 'dst': 28}
{'phrase': '言っ', 'dst': 23}
{'phrase': 'ていまいが', 'dst': 25}
{'phrase': 'そこまで', 'dst': 25}
{'phrase': '問題ないよね、と', 'dst': 26}
{'phrase': '思うに', 'dst': 27}
{'phrase': '至った', 'dst': 28}
{'phrase': '次第です。', 'dst': -1}


In [10]:
# pydotで有向グラフとして画像保存
if len(edges) > 0:
    graph = pydot.graph_from_edges(edges, directed=True)
    graph.write_png('044.dot.png')